In [2]:
library(Seurat)
library(tidyverse)
library(cowplot)
library(patchwork)
library(ggplot2)
library(readxl)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: 'SeuratObject'


The following objects are masked from 'package:base':

    intersect, t


-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'cowplot'


The following object is masked from 'package:lubridate':

    stamp



Attaching package: 'patchwork'


The following object is masked from 'package:cowplot':

    align_plots




In [3]:
setwd("/Users/alicedecugis/Desktop/capstone")
wt1 <- Read10X(data.dir = "/Users/alicedecugis/Desktop/capstone/GSE178693_RAW/WT1")
wt2 <- Read10X(data.dir = "/Users/alicedecugis/Desktop/capstone/GSE178693_RAW/WT2")
mutant1 <- Read10X(data.dir = "/Users/alicedecugis/Desktop/capstone/GSE178693_RAW/Mutant1")
mutant2 <- Read10X(data.dir = "/Users/alicedecugis/Desktop/capstone/GSE178693_RAW/Mutant2")

pca_result_wt1 <- readRDS("/Users/alicedecugis/Desktop/capstone/pca_result.rds")
pca_result_wt2 <- readRDS("/Users/alicedecugis/Desktop/capstone/pca_result_wt2.rds")
pca_result_m1 <- readRDS("/Users/alicedecugis/Desktop/capstone/pca_result_m1.rds")
pca_result_m2 <- readRDS("/Users/alicedecugis/Desktop/capstone/pca_result_m2.rds")

marker_table <- read_excel("/Users/alicedecugis/Desktop/capstone/cell_types.xlsx", col_names = TRUE, skip = 1)
colnames(marker_table)

In [5]:
# Load necessary libraries
library(Seurat)
library(readxl)

# Set working directory
setwd("/Users/alicedecugis/Desktop/capstone")

# Step 1: Load the PCA results
pca_result_wt1 <- readRDS("/Users/alicedecugis/Desktop/capstone/pca_result.rds")
pca_result_wt2 <- readRDS("/Users/alicedecugis/Desktop/capstone/pca_result_wt2.rds")
pca_result_m1 <- readRDS("/Users/alicedecugis/Desktop/capstone/pca_result_m1.rds")
pca_result_m2 <- readRDS("/Users/alicedecugis/Desktop/capstone/pca_result_m2.rds")

# Step 2: Load the marker table
marker_table <- read_excel("/Users/alicedecugis/Desktop/capstone/cell_types.xlsx", col_names = TRUE, skip = 1)

# Step 3: Create Seurat Objects for each condition (if not already done)
wt1 <- Read10X(data.dir = "/Users/alicedecugis/Desktop/capstone/GSE178693_RAW/WT1")
wt2 <- Read10X(data.dir = "/Users/alicedecugis/Desktop/capstone/GSE178693_RAW/WT2")
mutant1 <- Read10X(data.dir = "/Users/alicedecugis/Desktop/capstone/GSE178693_RAW/Mutant1")
mutant2 <- Read10X(data.dir = "/Users/alicedecugis/Desktop/capstone/GSE178693_RAW/Mutant2")

wt1_seurat <- CreateSeuratObject(counts = wt1, project = "WT1", min.cells = 3, min.features = 200)
wt2_seurat <- CreateSeuratObject(counts = wt2, project = "WT2", min.cells = 3, min.features = 200)
mutant1_seurat <- CreateSeuratObject(counts = mutant1, project = "Mutant1", min.cells = 3, min.features = 200)
mutant2_seurat <- CreateSeuratObject(counts = mutant2, project = "Mutant2", min.cells = 3, min.features = 200)


Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


In [6]:
# Step 4: Merge the Seurat objects into one combined dataset
combined <- merge(wt1_seurat, y = c(wt2_seurat, mutant1_seurat, mutant2_seurat), 
                  add.cell.ids = c("WT1", "WT2", "Mutant1", "Mutant2"))


In [8]:
# Step 4: Merge the Seurat objects into one combined dataset
combined <- merge(wt1_seurat, y = c(wt2_seurat, mutant1_seurat, mutant2_seurat), 
                  add.cell.ids = c("WT1", "WT2", "Mutant1", "Mutant2"))

# Step 5: Normalize the data
combined <- NormalizeData(combined)

# Step 6: Identify variable features
combined <- FindVariableFeatures(combined)

# Step 7: Scale the data
combined <- ScaleData(combined)

# Step 8: Perform PCA
combined <- RunPCA(combined, features = VariableFeatures(object = combined))

# Step 9: Find neighbors and perform clustering (Louvain clustering)
combined <- FindNeighbors(combined, dims = 1:20)
combined <- FindClusters(combined, resolution = 0.5)

# Step 10: Run UMAP for dimensionality reduction and visualization
combined <- RunUMAP(combined, dims = 1:20)

Normalizing layer: counts.WT1

Normalizing layer: counts.WT2

Normalizing layer: counts.Mutant1

Normalizing layer: counts.Mutant2

Finding variable features for layer counts.WT1

Finding variable features for layer counts.WT2

Finding variable features for layer counts.Mutant1

Finding variable features for layer counts.Mutant2

Centering and scaling data matrix

PC_ 1 
Positive:  Flt1, Ly6c1, Ly6a, Slco1a4, Abcb1a, Apold1, Gpr116, Itm2a, Klf2, Eltd1 
	   Igfbp7, Slco1c1, Ptprb, Rgs5, Bsg, Ly6e, Ly6c2, Palmd, Abcg2, Sdpr 
	   Egfl7, Id1, Pcp4l1, 9430020K01Rik, Cldn5, Klf4, Slc2a1, Lef1, Rundc3b, Hspb1 
Negative:  Aldoc, Slc1a2, Dbi, Ttyh1, Slc4a4, Slc1a3, Ndrg2, Pla2g7, Scg3, Gria4 
	   Gpr37l1, Luzp2, Aqp4, Gria1, Ednrb, Clu, Apoe, Fam213a, Timp4, Gm3764 
	   Ntsr2, Grid2, Gja1, Mt3, Ntrk2, Sdc4, Bai3, Mlc1, Mt1, Gnao1 
PC_ 2 
Positive:  Ctss, C1qb, Laptm5, C1qa, C1qc, Csf1r, Tyrobp, Plek, Fcer1g, Ccl4 
	   Nfkbia, Cd83, Cx3cr1, Ccl3, Siglech, Srgn, Cd14, Aif1, Selplg, Ptprc 
	   Tnf

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 6481
Number of edges: 237554

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9330
Number of communities: 16
Elapsed time: 0 seconds


Warning message:
"The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session"
14:56:52 UMAP embedding parameters a = 0.9922 b = 1.112

14:56:52 Read 6481 rows and found 20 numeric columns

14:56:52 Using Annoy for neighbor search, n_neighbors = 30

14:56:52 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

14:56:52 Writing NN index file to temp file /var/folders/f4/sv9t7hdn50z1_5ws19zm4bpm0000gn/T//RtmpcQ9sP1/file36dd70a6b8d9

14:56:52 Searching Annoy index using 1 thread, search_k = 3000

14:56:54 Annoy recall = 100%

14:56:54 Commencing smooth kNN distance calibrat

In [22]:
# Extract the group label (e.g., "WT1" before the first underscore)
combined$group_label <- gsub("(^[A-Za-z0-9]+)_.*", "\\1", combined$orig.ident)

# Set the group labels as identities in Seurat
Idents(combined) <- combined$group_label

# Check the distribution of groups
table(combined$group_label)



Mutant1 Mutant2     WT1     WT2 
   1323    2393    1891     874 

In [23]:
# Make sure you're working with the correct assay, such as "RNA" or "integrated"
DefaultAssay(combined) <- "RNA"  # Choose the correct assay (RNA, integrated, etc.)

# Run FindMarkers after setting the DefaultAssay
#degs_wt_vs_mutant <- FindMarkers(combined, ident.1 = "WT1", ident.2 = "Mutant1", test.use = "wilcox")


In [24]:
# Normalize each dataset
wt1_seurat <- NormalizeData(wt1_seurat)
wt2_seurat <- NormalizeData(wt2_seurat)
mutant1_seurat <- NormalizeData(mutant1_seurat)
mutant2_seurat <- NormalizeData(mutant2_seurat)

# Find anchors for the integration
anchors <- FindIntegrationAnchors(
  object.list = list(wt1_seurat, wt2_seurat, mutant1_seurat, mutant2_seurat),
  dims = 1:30  # Adjust the number of dimensions if necessary
)


Normalizing layer: counts

Normalizing layer: counts

Normalizing layer: counts

Normalizing layer: counts

Warning message in CheckDuplicateCellNames(object.list = object.list):
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."
Computing 2000 integration features

No variable features found for object1 in the object.list. Running FindVariableFeatures ...

Finding variable features for layer counts

No variable features found for object2 in the object.list. Running FindVariableFeatures ...

Finding variable features for layer counts

No variable features found for object3 in the object.list. Running FindVariableFeatures ...

Finding variable features for layer counts

No variable features found for object4 in the object.list. Running FindVariableFeatures ...

Finding variable features for layer counts

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors



In [25]:
# Check if layers are joined
combined@assays
combined$RNA$data <- combined@assays$RNA$data.WT1 # or "data.WT2", "data.Mutant1", etc.
combined <- IntegrateData(anchorset = anchors) # use pre-computed anchors

# Run differential expression analysis between WT1 and Mutant1
#degs_wt_vs_mutant <- FindMarkers(combined, ident.1 = "WT1", ident.2 = "Mutant1", test.use = "wilcox")


$RNA
Assay (v5) data with 18588 features for 6481 cells
First 10 features:
 Plekhg1, Mthfd1l, Akap12, Zbtb2, Rmnd1, Gm16153, 1700052N19Rik,
Ccdc170, Esr1, Syne1 
Layers:
 counts.WT1, counts.WT2, counts.Mutant1, counts.Mutant2, data.WT1,
scale.data.WT1, data.WT2, scale.data.WT2, data.Mutant1,
scale.data.Mutant1, data.Mutant2, scale.data.Mutant2 

$integrated
Assay data with 2000 features for 6481 cells
Top 10 variable features:
 Hbb-bs, Hba-a2, Hba-a1, Prph, Hbb-bt, Ccl3, Ctss, Syt4, Ywhah, Uchl1 


Merging dataset 2 into 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Warning message:
"Layer counts isn't present in the assay object; returning NULL"
Merging dataset 3 into 1 2

Extracting anchors for merged samples

Finding integration vectors

Warning message:
"Different cells in new layer data than already exists for scale.data"
Finding integration vector weights

Integrating data

Warning message:
"Layer counts isn't present in the assay object; returning NULL"
Merging dataset 4 into 1 2 3

Extracting anchors for merged samples

Finding integration vectors

Warning message:
"Different cells in new layer data than already exists for scale.data"
Finding integration vector weights

Integrating data

Warning message:
"Layer counts isn't present in the assay object; returning NULL"


In [ ]:
write.csv(degs_wt_vs_mutant, "/Users/alicedecugis/Desktop/capstone/degs_wt_vs_mutant.csv")

once it ran vizualise

In [30]:
degs_all <- FindAllMarkers(combined, test.use = "wilcox")
head(degs_all)

Calculating cluster WT1

Calculating cluster WT2

Calculating cluster Mutant1

Calculating cluster Mutant2



,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
Calca,0,1.4686284,0.020,0.866,0,WT1,Calca
Sncg,0,1.4475613,0.105,0.949,0,WT1,Sncg
Wdr78,0,0.5219852,0.024,0.864,0,WT1,Wdr78
Cd24a,0,1.3147479,0.044,0.883,0,WT1,Cd24a
Rab15,0,1.5124491,0.015,0.847,0,WT1,Rab15
3010026O09Rik,0,0.8695313,0.016,0.841,0,WT1,3010026O09Rik


In [31]:
head(degs_all)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
Calca,0,1.4686284,0.020,0.866,0,WT1,Calca
Sncg,0,1.4475613,0.105,0.949,0,WT1,Sncg
Wdr78,0,0.5219852,0.024,0.864,0,WT1,Wdr78
Cd24a,0,1.3147479,0.044,0.883,0,WT1,Cd24a
Rab15,0,1.5124491,0.015,0.847,0,WT1,Rab15
3010026O09Rik,0,0.8695313,0.016,0.841,0,WT1,3010026O09Rik


In [34]:
write.csv(degs_all, "/Users/alicedecugis/Desktop/capstone/degs_all.csv")